[View in Colaboratory](https://colab.research.google.com/github/Ritesh0100/MLtries/blob/master/TensorFlow_IrisClassification.ipynb)

In [0]:
#Getting started with graph execution - Iris Classification Problem
#https://www.tensorflow.org/get_started/get_started_for_beginners : using Estimator and DataSet_API

TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

In [0]:
import tensorflow as tf
import pandas as pd
import argparse #not used here

In [0]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']

In [0]:
#Loading and parsing data; tf.keras is a TensorFlow implementation of Keras
def load_data(label_name='Species'):
   
    # Create a local copy of the training set.
    train_path = tf.keras.utils.get_file(fname=TRAIN_URL.split('/')[-1],
                                         origin=TRAIN_URL)
    # train_path now holds the pathname: ~/.keras/datasets/iris_training.csv

    
    # Parse the local CSV file to Data frame
    train = pd.read_csv(filepath_or_buffer=train_path,
                        names=CSV_COLUMN_NAMES,  # list of column names
                        header=0  # ignore the first row of the CSV file.
                       )
    
    #dividing Data to features and label
    train_features, train_label = train, train.pop(label_name)

    # Apply the preceding logic to the test set.
    test_path = tf.keras.utils.get_file(TEST_URL.split('/')[-1], TEST_URL)
    test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
    test_features, test_label = test, test.pop(label_name)

    # Return four DataFrames.
    return (train_features, train_label), (test_features, test_label)

In [0]:
# Call load_data() to parse the CSV file.
(train_feature, train_label), (test_feature, test_label) = load_data()


In [0]:
# Create feature columns for all features; telling type of data to expect
train_x = train_feature
my_feature_columns = []
for key in train_x.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

In [14]:
#making Neural Net Model
classifier = tf.estimator.DNNClassifier(
        feature_columns=my_feature_columns,
        hidden_units=[10, 10],
        n_classes=3) #here we didn't specify optimizer 

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpvuuok3e_', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb64dc9fda0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
def train_input_fn(features, labels, batch_size):
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
  dataset = dataset.shuffle(buffer_size=1000).repeat(count=None).batch(batch_size)
  #Setting the buffer_size to a value larger than the number of examples (120) ensures that the data will be well shuffled
  return dataset.make_one_shot_iterator().get_next()

In [22]:
classifier.train(
        input_fn=lambda:train_input_fn(train_feature, train_label, 1000),steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpvuuok3e_/model.ckpt.
INFO:tensorflow:loss = 2542.9434, step = 1
INFO:tensorflow:global_step/sec: 122.742
INFO:tensorflow:loss = 169.52368, step = 101 (0.822 sec)
INFO:tensorflow:global_step/sec: 219.923
INFO:tensorflow:loss = 96.872284, step = 201 (0.453 sec)
INFO:tensorflow:global_step/sec: 198.272
INFO:tensorflow:loss = 75.48589, step = 301 (0.505 sec)
INFO:tensorflow:global_step/sec: 219.606
INFO:tensorflow:loss = 67.08708, step = 401 (0.453 sec)
INFO:tensorflow:global_step/sec: 216.711
INFO:tensorflow:loss = 60.53917, step = 501 (0.465 sec)
INFO:tensorflow:global_step/sec: 224.785
INFO:tensorflow:loss = 56.636253, step = 601 (0.438 sec)
INFO:tensorflow:global_step/sec: 223.563
INFO:tensorflow:los

In [0]:
#Evaluating model performance
def eval_input_fn(features, labels=None, batch_size=None):
    """An input function for evaluation or prediction"""
    features = dict(features)
    if labels is None:
        # No labels, use only features.
        inputs = features
    else:
        inputs = (features, labels)

    # Convert inputs to a tf.dataset object.
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)

    # Return the read end of the pipeline.
    return dataset.make_one_shot_iterator().get_next()

In [75]:
# Evaluate the model.

eval_result = classifier.evaluate(
    input_fn=lambda:eval_input_fn(test_feature, test_label, 1000))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-30-12:58:28
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpvuuok3e_/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-06-30-12:58:28
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.93333334, average_loss = 0.070706174, global_step = 1000, loss = 2.1211853
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: /tmp/tmpvuuok3e_/model.ckpt-1000

Test set accuracy: 0.933



In [0]:
#using model
predict_x = {
        'SepalLength': [5.1, 5.9, 6.9],
        'SepalWidth': [3.3, 3.0, 3.1],
        'PetalLength': [1.7, 4.2, 5.4],
        'PetalWidth': [0.5, 1.5, 2.1],
    }

In [0]:
predictions = classifier.predict(
    input_fn=lambda:eval_input_fn(predict_x,
                                  labels=None,
                                  batch_size=1000))

expected = ['Setosa', 'Versicolor', 'Virginica']

In [83]:
for pred_dict, expec in zip(predictions, expected):
    template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')

    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]
    print(template.format(expected[class_id], 100 * probability, expec))


Prediction is "Versicolor" (99.8%), expected "Setosa"

Prediction is "Virginica" (99.0%), expected "Versicolor"
